In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import transformers
from datasets import load_dataset
import huggingface_hub
import wandb

wandb.login()
huggingface_hub.login()
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
# model_id = "DavidFM43/bertin-gpt-j-6b-half-sharded"
model_id = "bertin-project/bertin-gpt-j-6B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision="half",
    load_in_8bit=True,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
new_tokens = ["<SP>", "<EP>"]

num_added_toks = tokenizer.add_tokens(new_tokens)
print("We have added", num_added_toks, "tokens")
# Notice: resize_token_embeddings expect to receive the full size of the new vocabulary, i.e., the length of the tokenizer.
model.resize_token_embeddings(len(tokenizer))


In [ ]:
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model


model = prepare_model_for_int8_training(model)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


In [ ]:
dataset_id = "hackathon-somos-nlp-2023/podcasts-ner-es"
dataset = load_dataset(dataset_id, use_auth_token=True)


In [ ]:
def format_ds(example):
    text = example["text"]
    ents = []
    for ent in example["annotation"]:
        label = ent["label"]
        ent_text = text[ent["start"] : ent["end"]]
        ents.append(f"({label}, {ent_text})")
    entities_text = ", ".join(ents)
    example["prompt"] = f"<SP> text: {text}\n\n entities: {entities_text} <EP>"

    return example


dataset = dataset.map(format_ds, remove_columns=["id", "text", "annotation"])
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)


In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=8,
        warmup_steps=100,
        max_steps=1000,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=5,
        output_dir="outputs",
        report_to="wandb",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

trainer.train()

model.push_to_hub(
    "hackathon-somos-nlp-2023/bertin-gpt-j-6b-ner-es", use_auth_token=True
)
tokenizer.push_to_hub(
    "hackathon-somos-nlp-2023/bertin-gpt-j-6b-ner-es", use_auth_token=True
)
